# Quick Start Guide - Azure AI Foundry

This notebook provides a hands-on introduction to Azure AI Foundry. You'll learn how to:
1. Initialize the AI Project client
2. List available models
3. Create a simple chat completion request

## Prerequisites
- Completed environment setup from previous notebook
- Azure credentials configured

## Import Required Libraries and Setup

In the next cell, we'll:
1. Import the necessary Azure SDK libraries for authentication and AI Projects
2. Import standard Python libraries for environment variables and JSON handling
3. Initialize Azure credentials using DefaultAzureCredential
   - This will automatically use your logged-in Azure CLI credentials
   - Alternatively, it can use other authentication methods like environment variables or managed identity


In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
import os
import json

# Initialize credentials
credential = DefaultAzureCredential()

In [ ]:
import os
import json


def find_file(filename, search_path):
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

# Search in the parent directory and its subdirectories
parent_dir = os.path.dirname(os.getcwd())  # Get parent directory
file_path = find_file('cred.json', parent_dir)

print(f"Found cred.json at: {file_path}")

try:
    with open(file_path, 'r') as f:
        loaded_config = json.load(f)
    
    # Print the values to verify
    print("Project Connection String:", loaded_config['PROJECT_CONNECTION_STRING'])
    print("Tenant ID:", loaded_config['TENANT_ID'])
    print("Model Deployment ID:", loaded_config['MODEL_DEPLOYMENT_NAME'])
    
except FileNotFoundError:
    print(f"Could not find file at: {file_path}")
except json.JSONDecodeError:
    print(f"File exists but contains invalid JSON")


## Initialize AI Project Client

> **Note:** Before proceeding, ensure you:
> 1. Configure cred.json with your credentials
> 2. Update the project connection string in your `cred.json` file
> 3. Have a Hub and Project already provisioned in Azure AI Foundry

You can find your project connection string in [Azure AI Foundry](https://ai.azure.com) under your project's settings:

<img src="proj-conn-string.png" alt="Project Connection String Location" width="600"/>



## Creating the AI Project Client

In the next cell, we'll create an AI Project client using the connection string from our `cred.json` file.
> **Note:** This example uses the synchronous client. For higher performance scenarios, you can also create an asynchronous client by importing `asyncio` and using the async methods from `AIProjectClient`.

The client will be used to:
- Connect to your Azure AI Project using the connection string
- Authenticate using Azure credentials
- Enable making inference requests to your deployed models


In [ ]:
from dotenv import load_dotenv
from pathlib import Path


try:
    client = AIProjectClient.from_connection_string(
        conn_str=loaded_config["PROJECT_CONNECTION_STRING"],
        credential=credential
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")

## Create a Simple Completion
Let's try a basic completion request:

Now that we have an authenticated client, let's use it to make a chat completion request.
The code below demonstrates how to:
1. Get a ChatCompletionsClient from the azure-ai-inference package
2. Use it to make a simple completion request

We'll use the MODEL_DEPLOYMENT_NAME from our `cred.json` file, making it easy to switch between different
deployed models without changing code. This could be an Azure OpenAI model, Microsoft model, or other providers
that support chat completions.

> Note: Make sure you have the azure-ai-inference package installed (from requirements.txt or as mentioned in [README.md](../README.md#-quick-start))


In [ ]:
from azure.ai.inference.models import UserMessage

model_deployment_name = loaded_config['MODEL_DEPLOYMENT_NAME']

try:
    chat_client = client.inference.get_chat_completions_client()
    response = chat_client.complete(
        model=model_deployment_name, 
        messages=[UserMessage(content="How to be healthy in one sentence?")]
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {str(e)}")